# Assignment

In this assignment we will explore the building blocks required to create a contracting-expanding convolutional neural network (CNN) to perform brain tumor segmentation on MRI.

This tutorial is part of the class **Introduction to Deep Learning for Medical Imaging** at University of California Irvine (CS190); more information can be found at: https://github.com/peterchang77/dl_tutor/tree/master/cs190.

### Submission

Once complete, the following items must be submitted:

* final `*.ipynb` notebook
* final trained `*.hdf5` model file
* final compiled `*.csv` file with performance statistics

# Google Colab

The following lines of code will configure your Google Colab environment for this assignment.

### Enable GPU runtime

Use the following instructions to switch the default Colab instance into a GPU-enabled runtime:

```
Runtime > Change runtime type > Hardware accelerator > GPU
```

# Environment

### Jarvis library

In this notebook we will Jarvis, a custom Python package to facilitate data science and deep learning for healthcare. Among other things, this library will be used for low-level data management, stratification and visualization of high-dimensional medical data.

In [2]:
# --- Install jarvis (only in Google Colab or local runtime)
% pip install jarvis-md

     |████████████████████████████████| 81kB 5.5MB/s 
     |████████████████████████████████| 645kB 12.8MB/s 
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


### Imports

Use the following lines to import any additional needed libraries:

In [3]:
import os, numpy as np, pandas as pd
from tensorflow import losses, optimizers
from tensorflow.keras import Input, Model, models, layers
from jarvis.train import datasets, custom

# Data

The data used in this assignment will consist of brain tumor MRI exams derived from the MICCAI Brain Tumor Segmentation Challenge (BRaTS). More information about he BRaTS Challenge can be found here: http://braintumorsegmentation.org/. Each single 2D slice will consist of one of four different sequences (T2, FLAIR, T1 pre-contrast and T1 post-contrast).

The following lines of code will:

1. Download the dataset (if not already present) 
2. Prepare the necessary Python generators to iterate through dataset
3. Prepare the corresponding Tensorflow Input(...) objects for model definition

In [4]:
# --- Download dataset
datasets.download(name='mr/brats-2020-mip')

# --- Prepare generators and model inputs
configs = {'specs': {'ys': {'tumor': {'norms': {'clip': {'max': 1}}}}}}
gen_train, gen_valid, client = datasets.prepare(name='mr/brats-2020-mip', keyword='mip*vox', configs=configs)
inputs = client.get_inputs(Input)

[ 2021-05-04 04:50:54 ] [====================] 100.000% : Extracting archive (0000750 / 0000750) 

# Training

In this assignment we will train a standard 2D U-Net for brain tumor segmentation. In addition to the baseline U-Net architecture, at minimum **one** of the following modifications must be implemented:

* modification of the contracting / expanding backbone (e.g., incorporating ResNet, Inception, SENet, etc)
* modification of the skip connection to include residual connections
* modification of the skip connection to include additional convolution operations

You are also **encouraged** to try different permuations and customizations to achieve optimal validation accuracy.

### Define the model

In [5]:
# --- Define kwargs dictionary
# USE 3x3 FILTERS | BATCH NORM | ReLU | STRITED CONV
kwargs = {
    'kernel_size': (1, 3, 3),
    'padding': 'same'}

# --- Define lambda functions
conv = lambda x, filters, strides : layers.Conv3D(filters=filters, strides=strides, **kwargs)(x)
norm = lambda x : layers.BatchNormalization()(x)
relu = lambda x : layers.ReLU()(x)

# --- Define stride-1, stride-2 blocks
conv1 = lambda filters, x : relu(norm(conv(x, filters, strides=1)))
conv2 = lambda filters, x : relu(norm(conv(x, filters, strides=(1, 2, 2))))

In [6]:
# --- Define contracting layers
l1 = conv1(8, inputs['dat'])
l2 = conv1(16, conv2(16, l1))
l3 = conv1(32, conv2(32, l2))
l4 = conv1(48, conv2(48, l3))
l5 = conv1(64, conv2(64, l4))

# --- Define single transpose
tran = lambda x, filters, strides : layers.Conv3DTranspose(filters=filters, strides=strides, **kwargs)(x)

# --- Define transpose block
tran2 = lambda filters, x : relu(norm(tran(x, filters, strides=(1, 2, 2))))

# --- Define expanding layers
l6 = tran2(48, l5)

# --- Ensure shapes match
print(l4.shape)
print(l6.shape)

# --- Concatenate
concat = lambda a, b : layers.Concatenate()([a, b])
concat(l4, l6)

# layers for full expansion
l7  = tran2(32, conv1(48, concat(l4, l6)))
l8  = tran2(16, conv1(32, concat(l3, l7)))
l9  = tran2(8,  conv1(16, concat(l2, l8)))
l10 = conv1(8,  l9)

(None, None, 30, 30, 48)
(None, None, 30, 30, 48)


In [7]:
# --- Create logits
logits = {}
logits['tumor'] = layers.Conv3D(filters=2, name='tumor', **kwargs)(l10)

# --- Create model
model = Model(inputs=inputs, outputs=logits)

### Compile the model

In [8]:
# --- Compile model
model.compile(
    optimizer=optimizers.Adam(learning_rate=2e-4),
    loss={'tumor': losses.SparseCategoricalCrossentropy(from_logits=True)},
    metrics={'tumor': custom.dsc(cls=1)},
    experimental_run_tf_function=False)



In [9]:
# --- Load data into memory for faster training
client.load_data_in_memory()

[ 2021-05-04 04:52:01 ] [====================] 100.000% : Iterating | 000368    

### Train the model

In [10]:
# --- Train model
model.fit(
    x=gen_train, 
    steps_per_epoch=500, 
    epochs=4,
    validation_data=gen_valid,
    validation_steps=500,
    validation_freq=4,
    use_multiprocessing=True)

Epoch 1/4
500/500 [==============================] - 115s 155ms/step - loss: 0.2838 - dsc_1: 0.3727
Epoch 2/4
500/500 [==============================] - 79s 158ms/step - loss: 0.0284 - dsc_1: 0.7681
Epoch 3/4
500/500 [==============================] - 79s 158ms/step - loss: 0.0168 - dsc_1: 0.8019
Epoch 4/4
500/500 [==============================] - 152s 303ms/step - loss: 0.0128 - dsc_1: 0.8244 - val_loss: 0.0117 - val_dsc_1: 0.8143


# Evaluation

Based on the tutorial discussion, use the following cells to check your algorithm performance. Consider loading a saved model and running prediction using `model.predict(...)` on the data aggregated via a test generator.

The Dice score values should be calculated for the foreground class (tumor); the Dice score for background does not need to be evaluated. As discussed in the tutorial, accuracy is determined on a patient by patient (volume by volume) basis, so please calculate the Dice score values on the entire 3D volume (not slice-by-slice).

### Performance

The following minimum performance metrics must be met for full credit:

* tumor: mean Dice score > 0.70

In [11]:
def dice(y_true, y_pred, c=1, epsilon=1):
    """
    Method to calculate the Dice score coefficient for given class
    
    :params
    
      (np.ndarray) y_true : ground-truth label
      (np.ndarray) y_pred : predicted logits scores
      (int)             c : class to calculate DSC on
    
    """
    assert y_true.ndim == y_pred.ndim
    
    true = y_true[..., 0] == c
    pred = np.argmax(y_pred, axis=-1) == c 

    A = np.count_nonzero(true & pred) * 2
    B = np.count_nonzero(true) + np.count_nonzero(pred) + epsilon
    
    return A / B

# --- Create validation generator
test_train, test_valid = client.create_generators(test=True, expand=True)
x, y = next(test_train)
logits = model.predict(x['dat'])

dsc = []

for x, y in test_valid:
    
    # --- Predict
    logits = model.predict(x['dat'])

    if type(logits) is dict:
        logits = logits['tumor']

    # --- Argmax
    dsc.append(dice(y['tumor'][0], logits[0], c=1))

dsc = np.array(dsc)

[ 2021-05-04 05:00:16 ] [====================] 100.000% : Iterating | 000074    

# New Section

**Note**: this cell is used only to check for model performance prior to submission. It will not be graded. Once submitted, your model will be benchmarked against the (same) validation cohort to determine final algorithm performance and grade. If your evaluation code above is correct the algorithm accuracy should match and you can confident that you will recieve full credit for the assignment. Once you are satisfied with your model, proceed to submission of your assignment below.

### Results

When ready, create a `*.csv` file with your compiled **validation** cohort Dice score statistics. There is no need to submit training performance accuracy. As in the tutorial, ensure that the tumor Dice score calculation column is present in the `*.csv` file.

In [12]:
# --- Define columns
df = pd.DataFrame(index=np.arange(dsc.size))
df['Dice score'] = dsc

# --- Print accuracy
print(df['Dice score'].mean())
# --- Create *.csv

# --- Serialize *.csv
df.to_csv('./results.csv')

0.8021811757942301


# Submission

Use the following line to save your model for submission (in Google Colab this should save your model file into your personal Google Drive):

In [13]:
# --- Serialize a model
model.save('./segmentation.hdf5')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Canvas

Once you have completed this assignment, download the necessary files from Google Colab and your Google Drive. You will then need to submit the following items:

* final (completed) notebook: `[UCInetID]_assignment.ipynb`
* final (results) spreadsheet: `[UCInetID]_results.csv`
* final (trained) model: `[UCInetID]_model.hdf5`

**Important**: please submit all your files prefixed with your UCInetID as listed above. Your UCInetID is the part of your UCI email address that comes before `@uci.edu`. For example, Peter Anteater has an email address of panteater@uci.edu, so his notebooke file would be submitted under the name `panteater_notebook.ipynb`, his spreadshhet would be submitted under the name `panteater_results.csv` and and his model file would be submitted under the name `panteater_model.hdf5`.